#### Задание 1
Сгенерировать последовательности, которые бы состояли из цифр (от 0 до 9) и задавались следующим образом:   
x - последовательность цифр   
y1 = x1, y(i) = x(i) + x(1). Если y(i) >= 10, то y(i) = y(i) - 10

**Задача:**
1. научить модель предсказывать y(i) по x(i)
2. пробовать RNN, LSTM, GRU

#### Задание 2 (дополнительное и необязательное)
применить LSTM для решения лекционного практического [задания](colab.research.google.com...mRGTSSeICL)

In [32]:
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

from sklearn.model_selection import train_test_split

##### 1. Генерация данных

In [4]:
def generate_sequence(length):
    x = np.random.randint(0, 10, length)
    y = np.zeros_like(x)
    y[0] = x[0]
    for i in range(1, length):
        y[i] = x[i] + x[0]
        if y[i] >= 10:
            y[i] -= 10
    return x, y

In [11]:
# Генерация набора данных
data_size = 10000
sequence_length = 10
X = []
Y = []
for _ in range(data_size):
    x, y = generate_sequence(sequence_length)
    X.append(x)
    Y.append(y)

X = np.array(X, dtype=int)
Y = np.array(Y, dtype=int)


In [12]:
# Посмотрим получившыйся набор данных Х
X

array([[5, 4, 8, ..., 3, 9, 8],
       [2, 9, 8, ..., 2, 2, 8],
       [9, 2, 6, ..., 6, 8, 9],
       ...,
       [0, 7, 4, ..., 5, 9, 9],
       [1, 7, 2, ..., 5, 2, 1],
       [6, 7, 5, ..., 1, 4, 9]], shape=(10000, 10))

In [13]:
# Посмотрим получившыйся набор данных Y
Y

array([[5, 9, 3, ..., 8, 4, 3],
       [2, 1, 0, ..., 4, 4, 0],
       [9, 1, 5, ..., 5, 7, 8],
       ...,
       [0, 7, 4, ..., 5, 9, 9],
       [1, 8, 3, ..., 6, 3, 2],
       [6, 3, 1, ..., 7, 0, 5]], shape=(10000, 10))

##### 2. Подготовка данных
Разделим данные на обучающую и тестовую выборки и преобразуем их в тензоры PyTorch.

In [14]:
# Разделение данных на обучающую и тестовую выборки
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# Преобразование данных в тензоры Pytorch
X_train = torch.Tensor(X_train).unsqueeze(-1) # Добавляем размерность для признаков
X_test = torch.Tensor(X_test).unsqueeze(-1)
Y_train = torch.Tensor(Y_train) 
Y_test = torch.Tensor(Y_test)

In [15]:
# Создадим DataLoader
train_dataset = TensorDataset(X_train, Y_train)
test_dataset = TensorDataset(X_test, Y_test)

batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

#### 3. Создание модели
Создадим модели RNN, LSTM и GRU

In [64]:
# Создадим модель RNN
class RNN_Model(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN_Model, self).__init__()
        self.rnn = nn.RNN(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        h0 = torch.zeros(1, x.size(0), self.rnn.hidden_size)
        out, _ = self.rnn(x, h0)
        out = self.fc(out)
        return out
  
# Создадим модель LSTM
class LSTM_Model(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers=1):
        super(LSTM_Model, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        h0 = torch.zeros(1, x.size(0), self.lsrm.hidden_size).to(x.device)
        c0 = torch.zeros(1, x.size(0), self.lstm.nidden_size).to(x.device)
        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out)
        return out

# Создадим модель GRU 
class GRU_Model(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(GRU_Model, self).__init__()
        self.gru = nn.GRU(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        h0 = torch.zeros(1, x.size(0), self.gru.hidden_size).to(x.device)
        out, _ = self.gru(x, h0)
        out = self.fc(out)
        return out

In [60]:
# Параметры модели
input_size = 1
hidden_size = 50
output_size = 10
num_layers = 1

In [61]:
# Инициализируем модели
rnn_model = RNN_Model(input_size, hidden_size, output_size)
lstm_model = LSTM_Model(input_size, hidden_size, output_size)
gru_model = GRU_Model(input_size, hidden_size, output_size) 

#### 4. Обучение моделей

In [62]:
# Функция потерь и оптимизатор
criterion = nn.MSELoss()
learning_rate = 0.001
num_epoch = 20

In [65]:
# Обучим модель
def train_model(model, train_loader, num_epochs=num_epoch):
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    for epoch in range(num_epoch):
        model.train()
        for X_batch, y_batch in train_loader:
            optimizer.zero_grad()
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)
            loss.backward()
            optimizer.step()
        print(f"Epoch [{epoch+1}/{num_epochs}], loss: {loss.item():.3f}")

print("Training RNN Model...")
train_model(rnn_model, train_loader)

print("Training LSTM Model...")
train_model(lstm_model, train_loader)

print("Training GRU Model...")
train_model(gru_model, train_loader)

Training RNN Model...


RuntimeError: The size of tensor a (10) must match the size of tensor b (32) at non-singleton dimension 1